In [8]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

load_dotenv()


True

In [20]:
llm = ChatOpenAI(model="gpt-4o-mini")

# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = WebBaseLoader(
    web_paths=("https://planetsurfcamps.co.uk/history-of-surfing/#:~:text=The%20first%20surfing%20references%20were,important%20part%20of%20the%20religion.",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("entry-content")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


In [32]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [39]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [45]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

### Statefully manage chat history ###
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)



In [50]:
conversational_rag_chain.invoke(
    {"input": "Where the people surf in the beginnings?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'In the beginnings, people surfed in Polynesia, as indicated by cave paintings. The sport was later brought to Hawaii by Polynesians, where it became an integral part of the culture.'

In [54]:
conversational_rag_chain.invoke(
    {"input": "And where is it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

[HumanMessage(content='Where the people surf in the beginnings?'), AIMessage(content='In the beginnings, people surfed in Polynesia, with early references found in cave paintings. Polynesians later brought surfing to Hawaii, where it became an important cultural and religious activity. Surfing was practiced by everyone in Hawaii, including men, women, children, and even kings.'), HumanMessage(content='How big are the waves there?'), AIMessage(content="The retrieved context does not provide specific information about the size of the waves in Hawaii or Polynesia. Therefore, I don't know the answer."), HumanMessage(content='And where is it?'), AIMessage(content='Hawaii is an archipelago located in the central Pacific Ocean, while Polynesia refers to a larger region in the Pacific that includes numerous islands, including Hawaii. Polynesia is situated southeast of Asia and northeast of Australia.'), HumanMessage(content='Where the people surf in the beginnings?'), AIMessage(content='In the

In [55]:
# Obtener el historial de mensajes de la sesión
messages = conversational_rag_chain.get_session_history("abc123").messages

# Formatear y presentar los mensajes
for i, message in enumerate(messages):
    if isinstance(message, HumanMessage):
        print(f"Human: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"AI: {message.content}")
    print()  # Añade una línea en blanco entre cada par de mensajes

Human: Where the people surf in the beginnings?

AI: In the beginnings, people surfed in Polynesia, with early references found in cave paintings. Polynesians later brought surfing to Hawaii, where it became an important cultural and religious activity. Surfing was practiced by everyone in Hawaii, including men, women, children, and even kings.

Human: How big are the waves there?

AI: The retrieved context does not provide specific information about the size of the waves in Hawaii or Polynesia. Therefore, I don't know the answer.

Human: And where is it?

AI: Hawaii is an archipelago located in the central Pacific Ocean, while Polynesia refers to a larger region in the Pacific that includes numerous islands, including Hawaii. Polynesia is situated southeast of Asia and northeast of Australia.

Human: Where the people surf in the beginnings?

AI: In the beginnings, people surfed in Polynesia, with early references found in cave paintings. Polynesians later brought surfing to Hawaii, wh